In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''http://medicaldecathlon.com/

'''

'http://medicaldecathlon.com/\n\n'

In [1]:
# %time
# !tar -xvf "/content/drive/Shareddrives/courseDrive/16/Task03_Liver.tar" -C "/content/drive/Shareddrives/TeamDriveForJmi/datasets/Liver_tumor/" 

In [ ]:
# 1jyVGUGyxKBXV6_9ivuZapQS8eUJXCIpu

In [ ]:
!pip install celluloid

##Data visualization

In [ ]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from celluloid import Camera
from IPython.display import HTML


In [ ]:
root = Path("/content/drive/Shareddrives/TeamDriveForJmi/datasets/Liver_tumor/Task03_Liver/imagesTr/")
label = Path("/content/drive/Shareddrives/TeamDriveForJmi/datasets/Liver_tumor/Task03_Liver/labelsTr/")

In [ ]:
def change_img_to_label_path(path):
  parts = list(path.parts)
  parts[parts.index("imagesTr")] = "labelsTr"
  return Path(*parts)

In [ ]:
sample_path = list(root.glob("liver_*"))[0]
sample_path_label = change_img_to_label_path(sample_path)


In [ ]:
ct = nib.load(sample_path).get_fdata()
mask = nib.load(sample_path_label).get_fdata().astype(int)

In [ ]:
ct.shape

(512, 512, 685)

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
fig = plt.figure()
camera = Camera(fig)

for i in range(ct.shape[2]):
  plt.imshow(ct[:,:,i],cmap="bone")
  mask_ = np.ma.masked_where(mask[:,:,i]==0,mask[:,:,1])
  plt.imshow(mask_,alpha=0.5)
  camera.snap()
animation = camera.animate();

<IPython.core.display.Javascript object>

In [ ]:
### Define model


In [ ]:
import torch

In [ ]:
class DoubleConv(torch.nn.Module):

  def __init__(self,in_channels,out_channels):

    super().__init__()
    self.step = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels,out_channels,3,padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(out_channels,out_channels,3,padding=1),
        torch.nn.ReLU()
    )

  def forward(self,X):
    return self.step(X)
    

In [ ]:
class UNet(torch.nn.Module):
  def __init__(self):

    super().__init__()

    self.layer1 = DoubleConv(1,64),
    self.layer2 = DoubleConv(64,128),
    self.layer3 = DoubleConv(128,256),
    self.layer4 = DoubleConv(256,512),

    self.layer5 = DoubleConv(512+256,256),
    self.layer6 = DoubleConv(256+128,128),
    self.layer7 = DoubleConv(128+64,64),
    self.layer8 = torch.nn.Conv2d(64,1,1)
   

    self.maxpool = torch.nn.MaxPool2d(2)

  def forward(self,x):
    x1 = self.layer1(x)
    x1m = self.maxpool(x1)

    x2 = self.layer2(x1m)
    x2m = self.maxpool(x2)

    x3 = self.layer3(x2m)
    x3m = self.maxpool(x3)

    x4 = self.layer4(x3m)

    x5 = torch.nn.Upsample(scale_factor=2,mode="trilinear")(x4)
    x5 = torch.cat([x5,x3],dim=1)
    x5 = self.layer5(x5)

    x6 = torch.nn.Upsample(scale_factor=2,mode="trilinear")(x5)
    x6 = torch.cat([x6,x2],dim=1)
    x6 = self.layer6(x6)   

    x7 = torch.nn.Upsample(scale_factor=2,mode="trilinear")(x6)
    x7 = torch.cat([x7,x1],dim=1)
    x7 = self.layer7(x7) 

    return self.layer8(x7)

In [ ]:
model = UNet()


In [ ]:
model

In [ ]:
random_input = torch.randn(1,1,128,128,128)

In [ ]:
output = model(random_input)
assert output.shape == torch.Size([1,3,128,128,128])

In [ ]:
!pip install torchio

     |████████████████████████████████| 164 kB 5.4 MB/s 
     |████████████████████████████████| 48.4 MB 3.2 MB/s 


In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 5.4 MB/s 
     |████████████████████████████████| 398 kB 44.1 MB/s 
     |████████████████████████████████| 134 kB 44.0 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.8 MB/s 
     |████████████████████████████████| 144 kB 41.7 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 36.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [ ]:
import torchio as tio
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
path = Path("/content/drive/Shareddrives/TeamDriveForJmi/datasets/Liver_tumor/Task03_Liver/imagesTr/")
subject_paths = list(path.glob("liver_*"))
subjects = []

for subject_path in subject_paths:
  label_path = change_img_to_label_path(subject_path)
  subject = tio.Subject({"CT":tio.ScalarImage(subject_path),"Label":tio.LabelMap(label_path)})
  print(subject)

  subjects.append(subject)



Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); images: 2)
Subject(Keys: ('CT', 'Label'); ima

In [ ]:
len(subjects)

In [ ]:
for subject in subjects:
  assert subject["CT"].orientation == ("R","A","S")


In [ ]:
from torchio.transforms.preprocessing.intensity.rescale import RescaleIntensity
from torchio.transforms.preprocessing.spatial.crop_or_pad import CropOrPad
process = tio.Compose([
                        tio.CropOrPad((256,256,200)),
                        tio.RescaleIntensity((-1,1))
])

augmentation = tio.RandomAffine(scales=(0.9,1.1),degrees=(-10,10))

val_transform = process
train_transform = tio.Compose([process,augmentation])

In [ ]:
from torchio.transforms import transform
train_dataset = tio.SubjectsDataset(subjects[:105],transform=train_transform)
val_dataset = tio.SubjectsDataset(subjects[105:],transform=val_transform)

sampler = tio.data.LabelSampler(patch_size=96,label_name="Label",label_probabilities={0:0.2,1:0.3,2:0.5})


In [ ]:
train_patches_queue = tio.Queue(
    train_dataset,
    max_length=40,
    samples_per_volume=5,
    sampler=sampler,
    num_workers=4
)
val_pathches_queue = tio.Queue(
    val_dataset,
    max_length=40,
    samples_per_volume=5,
    sampler=sampler,
    num_workers=4
)



In [ ]:
train_loader = torch.utils.data.DataLoader(train_patches_queue,batch_size=2,num_workers=0)
val_loader = torch.utils.data.DataLoader(val_pathches_queue,batch_size=2,num_workers=0)

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
# !cat /proc/meminfo

In [ ]:
class Segmenter(pl.LightningModule):
  def __init__(self):
    super().__init__()

    self.model = UNet()
    self.optimizer = torch.optim.Adam(self.model.parameters(),lr=0.0001)
    self.loss_fn = torch.nn.CrossEntropyLoss()

  def forward(self,data):
    return self.model(data)

  def training_step(self,batch,batch_idx):
    img = batch["CT"]["data"]
    mask = batch["Label"]["data"][:,0]
    mask = mask.long()

    pred = self(img)
    loss = self.loss_fn(pred,mask)
    self.log("Train Loss",loss)

  def configure_optimizer(self):
    return [self.optimizer]



In [ ]:
model = segmenter()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor="Val Loss",
    save_top_k=10,
    mode="min"
)

In [ ]:
trainer = pl.Trainer(gpus=1,
                     logger=TensorBoardLogger(save_dir="/content/drive/Shareddrives/TeamDriveForJmi/saved_models/TS/"),
                     log_every_n_steps=1,
                     callbacks=checkpoint_callback,
                     max_epochs=100)

In [ ]:
%time
trainer.fit(model,train_loader,val_loader)

# Evaluation

In [ ]:
model = Segmenter.load_from_checkpoints()
device = torch.device("cuda :0" if torch.cuda.is_available else "cpu")
model = model.eval()
model.to(device)



In [ ]:
IDX = 4
imgs = val_dataset[IDX]["CT"]["data"]
mask = val_dataset[IDX]["Label"]["data"]

grid_sampler = tio.inference.GridSampler(val_dataset[IDX],96,(8,8,8))


In [ ]:
aggregator = tio.inference.GridAggregator(grid_sampler)

patch_loader = torch.utils.data.Dataloader(grid_sampler,batch_size=4)


In [ ]:
with torch.no.grads():
  for patches_batch in patch_loader:
    input_tensor = patches_batch["CT"]["data"].to(device)
    locations = patches_batch[tio.LOCATION]
    pred = model(input_tensor)
    aggregator.add_batch(pred,locations)


In [ ]:
output_tensor = aggregator.get_output_tensor()

In [ ]:
fig = plt.figure()
camera = Camera(fig)
pred = output_tensor.argmax(0)


for i in range(0,output_tensor.shape[-1],2):
  plt.imshow(imgs[0,:,:,i],cmap="bone")
  mask = np.ma.masked_where(pred[:,:,i]==0,pred[:,:,i])
  plt.imshow(mask,alpha=0.5,cmap="autumn")
  camera.snap()

animation = camera


In [ ]:
HTML(animation.to_html5_video)